In [1]:
import cv2
import easyocr
import re

In [2]:
def readLargestPlateText(plateThresh, reader):
    result = reader.readtext(plateThresh)
    
    maxArea = 0
    largestText = 'No Detection'
    largestTextConfidence = 0
    
    for bbox, text, confidence in result:
        # Get box Coords
        x1, y1 = bbox[0]
        x2, y2 = bbox[-2]
        
        # Calculate area
        area = (x2 - x1) * (y2 - y1)
        
        # Update area if needed
        if area > maxArea:
            maxArea = area
            largestText = text.upper()
            largestTextConfidence = confidence
            
    # Remove non-alphanumeric characters
    cleanText = re.sub(r'[^a-zA-Z0-9]', '', largestText)
    
    return cleanText, largestTextConfidence

In [3]:
# Load image
plateImage = cv2.imread('test-media/test1.jpg')
grayImage = cv2.cvtColor(plateImage, cv2.COLOR_BGR2GRAY)

# Otsu thresholding
_, plateThresh = cv2.threshold(grayImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
plateThresh = cv2.morphologyEx(plateThresh, cv2.MORPH_CLOSE, kernel)

# Adaptive Threshold
plateThresh = cv2.adaptiveThreshold(
    plateThresh, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
    cv2.THRESH_BINARY, 21, 5)

# Setup OCR reader
reader = easyocr.Reader(['en'])

# Use the function to read the largest text region
plateText, confidence = readLargestPlateText(plateThresh, reader)

# Display results
print(f'Plate Text: {plateText}')
print(f'Confidence: {confidence}')
cv2.imshow("Thresholded Plate", plateThresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

Plate Text: XLTAKZH
Confidence: 0.19270969478583372
